In [28]:
"""
This is a notebook for producing predictions on a custom dataset, supplying confidence intervals for predictions in excel format. 
Note that this notebook does NOT supply accuracy values, but outputs raw predictions with a confidence interval

Data should be within a single directory i.e. test/image1...test/image2 etc.

Simply change the dataset download link in cell "Load custom data, then unzip and prepare data " & the unzip file name accordingly

This notebook uses a placeholder dataset composed from 20 Chevrolet Cobalt SS 2010 images from the cobalt test dataset

PLEASE ENSURE YOUR COLAB ENVIRONMENT USES A GPU ACCELERATOR


"""

import sys
import numpy as np
import string

#Load model from Notebook 2, saved in gdrive

!gdown https://drive.google.com/uc?id=1uegggLHdKPu11z_JQzzgkLPWnpGeqk_B
  


Downloading...
From: https://drive.google.com/uc?id=1uegggLHdKPu11z_JQzzgkLPWnpGeqk_B
To: /content/model_categorical_2setshalves_50epochs_10epochs.h5
30.5MB [00:00, 142MB/s] 


In [29]:
base_dir='model_categorical_2setshalves_50epochs_10epochs.h5'
import tensorflow as tf 
model = tf.keras.models.load_model(base_dir)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1311744   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 196)               200900    
Total params: 3,770,628
Trainable params: 3,716,980
Non-trainable params: 53,648
_________________________________________________________________
None


In [30]:
##########################FOR EVALUATION PURPOSES#####################################################

#Load car data, then unzip and prepare data 
#Replace this with your data for evaluation
!gdown https://drive.google.com/uc?id=1RZRdIB-M30XcH3unOd3VKbcbh_wHi0ix
  
#Change the path to your datapath. Ideally, provide it in zip format with a the folder directory predict->test->images

base_path='../content/predict'


Downloading...
From: https://drive.google.com/uc?id=1RZRdIB-M30XcH3unOd3VKbcbh_wHi0ix
To: /content/predict.zip
4.21MB [00:00, 65.6MB/s]


In [31]:
!unzip predict.zip


Archive:  predict.zip
replace predict/test/00850.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [32]:
#Training data for class reference only!

!gdown https://drive.google.com/uc?id=1VXAuj8P9FJaQK7bYFU2U3UwEYHCdRMvB


  
# Unzip train dataset
!unzip compressed.zip 


Downloading...
From: https://drive.google.com/uc?id=1VXAuj8P9FJaQK7bYFU2U3UwEYHCdRMvB
To: /content/compressed.zip
312MB [00:01, 186MB/s]
Archive:  compressed.zip
   creating: cropped_car_train/
   creating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/02603.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/04626.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/03998.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/03352.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/08119.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/03656.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/06165.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/06444.jpg  
  inflating: cropped_car_train/Chevrolet Malibu Hybrid Sedan 2010/06834.jpg  
  inflating: cropped_car_train/Chevro

In [54]:
import zipfile

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob, os, random
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt


#Creating datagenerators



train_path = '../content/cropped_car_train'



train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
   
    
)



train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    
    class_mode='categorical',
    subset='training',
    seed=0
)




test_datagen = ImageDataGenerator(
    rescale=1./255,
    
)


test_generator = test_datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=20,
    class_mode='categorical',
    shuffle=False,
    seed=0
)

test_generator.reset()


Found 6598 images belonging to 196 classes.
Found 20 images belonging to 1 classes.


In [55]:
#Predicting on custom dataset

#Define predict model first

pred=model.predict(test_generator,verbose=1,steps=len(test_generator))






1/1 [==============================] - 0s 312ms/step


In [62]:
print(pred)

print(type(pred))

[[4.1228194e-08 2.0793649e-04 6.3963649e-03 ... 1.3548161e-05
  1.4107803e-06 1.6772520e-06]
 [5.2203147e-10 8.2106465e-01 2.3526773e-08 ... 8.0819546e-11
  5.1180869e-13 1.6546002e-10]
 [8.0204789e-17 2.3689502e-06 2.1122487e-08 ... 3.6766680e-06
  9.4529572e-17 1.4930067e-14]
 ...
 [2.1380632e-12 2.3034936e-07 1.9505626e-10 ... 2.1998824e-08
  9.1466607e-10 6.6425376e-10]
 [5.2970275e-13 5.5919183e-05 2.8060285e-05 ... 2.0901831e-07
  4.7450742e-13 2.1889681e-11]
 [4.3694303e-08 3.5784082e-04 7.5735611e-06 ... 7.9053831e-07
  2.6079367e-12 1.4430408e-09]]
<class 'numpy.ndarray'>


In [0]:

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions,
                      "Probabilities":pred.tolist()
                    })

results.to_csv("output_results.csv")